## Register Model


In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [1]:
import torch
import torchvision
import numpy as np
from pathlib import *
from PIL import Image
from azureml.core.workspace import Workspace
from azureml.core.model import Model
from dotenv import set_key, find_dotenv
from testing_utilities import get_auth
import urllib

In [8]:
env_path = find_dotenv(raise_error_if_not_found=True)

### Model

We load a pretrained [**Mask R-CNN ResNet-50 FPN** object detection model](https://pytorch.org/blog/torchvision03/). This model is trained on subset of COCO train2017, which contains the same 20 categories as those from Pascal VOC.

In [ ]:
# use pretrained model: https://pytorch.org/blog/torchvision03/
model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)

#device = torch.device("cpu")
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

In [10]:
url = "https://download.pytorch.org/models/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth"
urllib.request.urlretrieve(url, "maskrcnn_resnet50.pth")

('maskrcnn_resnet50.pth', <http.client.HTTPMessage at 0x7f1553819048>)

In [9]:
img_path = "./test_image.jpg"
print(Image.open(img_path).size)
img = Image.open(img_path)

(1920, 1080)


In [12]:
img = np.array(img.convert(mode='RGB'), dtype = np.float32)  
img_tensor = torchvision.transforms.functional.to_tensor(img)/255

In [ ]:
model.eval()
with torch.no_grad():
    prediction = model([img_tensor.to(device)])

In [ ]:
print(prediction)

### Register Model



In [ ]:
# Get workspace
# Load existing workspace from the config file info.

ws = Workspace.from_config(auth=get_auth())
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep="\n")

In [4]:

model = Model.register(
    model_path="maskrcnn_resnet50.pth",  # this points to a local file
    model_name="maskrcnn_resnet50_model",  # this is the name the model is registered as
    tags={"model": "dl", "framework": "maskrcnn"},
    description="torchvision maskrcnn_resnet50",
    workspace=ws,
)

Registering model maskrcnn_resnet50_model


In [6]:
print(model.name, model.description, model.version)

maskrcnn_resnet50_model torchvision maskrcnn_resnet50 1


In [9]:

set_key(env_path, "model_version", str(model.version))

(True, 'model_version', '1')

Next we will proceed with notebook [032_DevelopModelDriver.ipynb](032_DevelopModelDriver.ipynb).